# Assignment Lesson 9:  Forecasting
In this assignment, we will explore the python package [statsmodels](http://www.statsmodels.org/stable/tsa.html) to forecast time series data. You will learn to use different time series modeling technique for forecasting.
<br>
Original version found in MLEARN 510 Canvas. Updated and modified by Ernst Henle
<br>
Copyright © 2024 by Ernst Henle 

# Learning Objectives:
- Decompose time series into autocorrelation, seasonality, trend, and noise. 
- Explain the effects of exponential smoothing models and differentiate them from other models.
- Apply and evaluate the results of an autoregressive model. 
- Apply and evaluate the results of a moving average model. 
- Apply and evaluate the results of an autoregressive integrated moving average model.
- Apply and evaluate the results of ARIMA model for forecasting (time series prediction).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')
# # Suppress the specific warnings
# warnings.filterwarnings("ignore", category=UserWarning, message="No frequency information was provided")
%matplotlib inline

## Air Passenger Dataset
This dataset provides monthly totals of international airline passengers from 1949 to 1960. You can find a copy of the dataset on [Kaggle](https://www.kaggle.com/rakannimer/air-passengers) or [R datasets](https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/AirPassengers.html).
1. The file is read in as a dataframe
2. The first column of the file is read in as an index of the dataframe
3. The index datatype is parsed as a datetime (`parse_dates=True`)
4. The index column header ('Month') is removed
5. The value column is called 'airline passengers'
6. The dataframe (144 rows) is split into training datframe (first 130 rows) and a testing dataframe (last 14 rows)

In [ ]:
# Read csv, use first column as index, parse 
df = pd.read_csv('airline-passengers.csv', index_col=[0], parse_dates=True)
df.index = df.index.values
display(df.head())

# split the data into train and test
train, test = df.iloc[:130, [0]], df.iloc[130:, [0]]
print(f'Data ({df.shape}) is split into training ({train.shape}) and  testing ({test.shape}) dataframes')

# Remove original data to avoid accidental usage
df = None

# Present the data
plt.plot(train)
plt.plot(test)
plt.show()

## Question 1.1
Using [seasonal_decompose](https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.seasonal_decompose.html) API from `statsmodels.tsa.seasonal`, apply additive decomposition to the training dataset and plot each component from the decomposition.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

additive_result = seasonal_decompose(train['airline passengers'], model='additive', period=12)
additive_result.plot()
plt.show()

## Question 1.2
Using [seasonal_decompose](https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.seasonal_decompose.html) API from `statsmodels.tsa.seasonal`, apply multiplicative decomposition to the same training dataset and plot each component from the decomposition. 

In [ ]:
multiplicative_result = seasonal_decompose(train['airline passengers'], model='multiplicative', period=12)
multiplicative_result.plot()
plt.show()

## Question 1.3
Determine the p-values of the [Augmented Dickey-Fuller test](https://www.statsmodels.org/stable/generated/statsmodels.tsa.stattools.adfuller.html) for the residuals of both the additive and multiplicative decompositions.

In [ ]:
from statsmodels.tsa.stattools import adfuller

adf_add = adfuller(additive_result.resid.dropna())
adf_mul = adfuller(multiplicative_result.resid.dropna())

print(f'Additive ADF p-value: {adf_add[1]:.4f}')
print(f'Multiplicative ADF p-value: {adf_mul[1]:.4f}')

## Question 1.4
Which decomposition makes more sense for this dataset?  Why? .
- Compare and discuss the two sets of decomposition plots
- Compare and discuss the two Augmented Dickey-Fuller test p-values
- Use 'Stationarity' to explain the value of the decompositions

The multiplicative decomposition is generally more appropriate. The magnitude of the seasonal component grows with the overall level, which suggests a multiplicative relationship. The ADF test on the multiplicative residuals typically yields a smaller p-value, indicating that the seasonally adjusted series is closer to stationary.

## Question 2.1
- Apply the simple exponential smoothing technique ([SimpleExpSmoothing](https://www.statsmodels.org/stable/generated/statsmodels.tsa.holtwinters.SimpleExpSmoothing.html)) to the airline dataset.
- Find the hyper-parameter `smoothing_level` (+/- 0.1) that has lowest RMSE.
- Report the prediction accuracy (RMSE) on the test dataset.
- Present the training, test, and predicted time series using the method `plotTrainTestPred`. 

In [ ]:
def plotTrainTestPred(train, test, pred):
    plt.plot(train['airline passengers'], label='train')
    plt.plot(test['airline passengers'], linewidth=3, label='test')
    plt.plot(pred, linestyle='--', label='predicted')
    plt.title(f'Compare Train, Test, and Predicted Time Series')
    plt.legend()
    plt.grid()
    plt.show();

In [ ]:
warnings.filterwarnings('ignore')

levels = np.arange(0.1, 1.1, 0.1)
rmse_vals = []
for lvl in levels:
    ses = SimpleExpSmoothing(train['airline passengers']).fit(smoothing_level=lvl, optimized=False)
    pred = ses.forecast(len(test))
    rmse = mean_squared_error(test['airline passengers'], pred, squared=False)
    rmse_vals.append(rmse)

best_idx = int(np.argmin(rmse_vals))
best_level = levels[best_idx]
best_rmse = rmse_vals[best_idx]

best_ses = SimpleExpSmoothing(train['airline passengers']).fit(smoothing_level=best_level, optimized=False)
ses_pred = best_ses.forecast(len(test))

print(f'Best smoothing_level: {best_level:.1f}  RMSE: {best_rmse:.2f}')

In [ ]:
plt.plot(levels, rmse_vals, marker='o')
plt.xlabel('smoothing_level')
plt.ylabel('RMSE')
plt.grid(True)
plt.show()

plotTrainTestPred(train, test, ses_pred)

## Question 2.2
Apply the HWES ([ExponentialSmoothing](https://www.statsmodels.org/stable/_modules/statsmodels/tsa/holtwinters/model.html)) technique to the airline dataset and report the prediction accuracy (RMSE) on the test dataset.
- Use the smoothing level from before.
- Use `trend` and `seasonal` hyper-parameters to improve model accuracy.

In [ ]:
warnings.filterwarnings('ignore')

trends = [None, 'add', 'mul']
seasonals = ['add', 'mul']
best_rmse = float('inf')
best_params = None
for tr in trends:
    for seas in seasonals:
        model = ExponentialSmoothing(train['airline passengers'], trend=tr, seasonal=seas, seasonal_periods=12)
        fit = model.fit(smoothing_level=best_level, optimized=True)
        pred = fit.forecast(len(test))
        rmse = mean_squared_error(test['airline passengers'], pred, squared=False)
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = (tr, seas, fit)

print(f'Best trend: {best_params[0]}, seasonal: {best_params[1]}, RMSE: {best_rmse:.2f}')
hw_pred = best_params[2].forecast(len(test))
plotTrainTestPred(train, test, hw_pred)

## Question 3
Apply Autoregressive (AR) model to the airline dataset and report the prediction accuracy (RMSE) on the test dataset. An AR model is a subset of the ARIMA [ARIMA](https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima.model.ARIMA.html), where only the `p` parameter of the `order=(p, d, q)` is used.
- Differencing `d` of the `order=(p, d, q)` is set to zero in AR.
- Lag `q` of the `order=(p, d, q)` is set to zero in AR.
- Find lag `p` of the `order=(p, d, q)` that minimizes RMSE. Try lags 10 through 22

In [ ]:
best_rmse = float('inf')
best_p = None
for p in range(10, 23):
    model = ARIMA(train, order=(p, 0, 0))
    fit = model.fit()
    pred = fit.forecast(len(test))
    rmse = mean_squared_error(test, pred, squared=False)
    if rmse < best_rmse:
        best_rmse = rmse
        best_p = p

print(f'Best p: {best_p}, RMSE: {best_rmse:.2f}')
ar_pred = ARIMA(train, order=(best_p, 0, 0)).fit().forecast(len(test))
plotTrainTestPred(train, test, ar_pred)

## Question 4
Apply Auto Regressive Moving Average (ARMA) model to the airline dataset and report the prediction accuracy (RMSE) on the test dataset. An ARMA model is a subset of [ARIMA](https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima.model.ARIMA.html), where only the `p` and `q` parameters of the `order=(p, d, q)` are used. 
- Set the `p` value of the `order=(p, d, q)` that you found for the AR model.
- Differencing `d` of the `order=(p, d, q)` is set to zero in ARMA.
- Find the lag `q` of the `order=(p, d, q)` that minimizes RMSE. Try values 10 through 22

In [ ]:
best_rmse = float('inf')
best_q = None
for q in range(10, 23):
    model = ARIMA(train, order=(best_p, 0, q))
    fit = model.fit()
    pred = fit.forecast(len(test))
    rmse = mean_squared_error(test, pred, squared=False)
    if rmse < best_rmse:
        best_rmse = rmse
        best_q = q

print(f'Best q: {best_q}, RMSE: {best_rmse:.2f}')
arma_pred = ARIMA(train, order=(best_p, 0, best_q)).fit().forecast(len(test))
plotTrainTestPred(train, test, arma_pred)

## Question 5
Apply Auto Regressive Integrated Moving Average model ([ARIMA](https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima.model.ARIMA.html)) to the airline dataset and report the prediction accuracy (RMSE) on the test dataset. In an ARIMA model we need to set the `p`, `d`, and `q` parameters of the `order=(p, d, q)` hyper parameter: 
- Set the `p` parameter of the `order=(p, d, q)` that you found for the AR model.
- Set the `q` parameter of the `order=(p, d, q)` that you found for the ARMA model.
- Optimize the ARIMA by finding the best `d` parameter of the `order=(p, d, q)` that minimizes RMSE:  try values 0, 1, and 2

In [ ]:
best_rmse = float('inf')
best_d = None
for d in [0, 1, 2]:
    model = ARIMA(train, order=(best_p, d, best_q))
    fit = model.fit()
    pred = fit.forecast(len(test))
    rmse = mean_squared_error(test, pred, squared=False)
    if rmse < best_rmse:
        best_rmse = rmse
        best_d = d

print(f'Best d: {best_d}, RMSE: {best_rmse:.2f}')
arima_pred = ARIMA(train, order=(best_p, best_d, best_q)).fit().forecast(len(test))
plotTrainTestPred(train, test, arima_pred)

## Question 6
After running through various time series models, summarize your findings. 

The airline passenger data shows a clear upward trend and seasonal pattern. Simple exponential smoothing captures the level but not the seasonality, while Holt-Winters and ARIMA models handle both components. In practice the Holt-Winters (HWES) or a tuned ARIMA model typically yields the lowest RMSE on the test set.